<a href="https://colab.research.google.com/github/huskeypm/cellmigration/blob/main/master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# master
Workhorse for generating and executing jobs in support of migration paper.

See https://github.com/huskeypm/cellmigration/blob/main/run/README.md
for how to run






In [3]:
%load_ext autoreload
%autoreload 2

In [5]:
!git clone https://github.com/huskeypm/cellmigration
#!cd cellmigration; git pull

fatal: destination path 'cellmigration' already exists and is not an empty directory.


In [6]:
import sys
sys.path.append("/content/cellmigration/")


from google.colab import drive
gDrive='/content/drive/'
drive.mount(gDrive)
gDrive += "MyDrive/temp/231212"




Mounted at /content/drive/


In [7]:
import yaml
import sys

sys.path.append('/content/cellmigration/paper/')
import writeInputs as wi
#import batchProcess as bp
#outDir = "/home/pkh-lab-shared/migration/231110/"
outDir = "/home/pkekeneshuskey/data/231212/"
outDir = gDrive
# FOR LOCAL
cmd = "python3 brownian_v3.py -yamlFile " # if local

# FOR FAUST/W
cmd='$EXEC ' # if remote host

wi.params = wi.Params(
    cmd=cmd,
    path=outDir
)

print(wi.params.cmd)
print(wi.params.path)

varIter = 7 # keys
varRuns = 5 # replicates per run



$EXEC 
/content/drive/MyDrive/temp/231212


The idea of the reference file is to have a complete system that toggled - for instance, we have a case that is tuned for ATP application, but we can set the ATP concentration to zero to emulate a ATP-free condition. Also want values that are intermediate, so when we vary them by 50%-200%, we get reasonable numbers

In [9]:
referenceFile="cellmigration/paper/template_full.yaml"
with open(referenceFile, 'r') as file:
     daVars = yaml.safe_load(file)

## crowder free case
### No ATP
Can copy/paste into file to run


In [10]:
nocrowders = daVars.copy()
nocrowders['nCrowders']=1
nocrowders['crowderRad']=0.1
nocrowders['outName']='nocrowder'
keys=['nCells','cellRad','cellAttr']
keys=['cellAttr']

#nocrowders

Make a case without a containment potential

In [ ]:
nocrowders_nocontainment = nocrowders.copy()
nocrowders_nocontainment['containmentPotential']=False
nocrowders_nocontainment['nUpdates']=10000
nocrowders_nocontainment['absorbingBoundary']=False
nocrowders_nocontainment['xPotential']=True
nocrowders_nocontainment['outName']= 'nocrowder_nocontainment'
key=['xScale']
varItersList = [0] # only one conc. makes sense ,1.,2.,3.,4.,5.,6.,7.]

daKey = key[0]
print("Generate ",daKey)
writeFile = nocrowders_nocontainment['outName']+"_"+daKey+".bash"
wi.WriteIterativeYaml(nocrowders_nocontainment,daKey,varIter=varItersList, varRuns=varRuns,
                      writeFile=writeFile)

Generate  xScale
Wrote nocrowder_nocontainment_xScale.bash


In [ ]:

####### cellAttr ----> particleAttr
for daKey in keys:
    print("Generate ",daKey)
    if "nCells" in daKey:
      varItersList = [30,40,50,60,70,80,90]
    elif "cellAttr" in daKey:
      varItersList = [0.5, 0.75 , 1.0, 2.5, 5.0]
    else:
      varItersList = varIter
    writeFile = nocrowders['outName']+"_"+daKey+".bash"
    wi.WriteIterativeYaml(nocrowders,daKey,varIter=varItersList, varRuns=varRuns,
                         writeFile=writeFile)


Generate  cellAttr
Wrote nocrowder_cellAttr.bash


In [ ]:
# generally too slow to run in notebook
#bp.processYamls(nocrowders['outName'],yamlNamePrefix=nocrowders['outName'],prefixOptions=keys,
#               display=False)

### + ATP



In [ ]:
nocrowdersatp = daVars.copy()
nocrowdersatp['nCrowders']=1
nocrowdersatp['crowderRad']=0.1
nocrowdersatp['outName']='nocrowder_atp'
nocrowdersatp['xPotential']=True
nocrowdersatp['yPotential']=True  # default yScale should be 0
keys=['nCells','cellRad','cellAttr','xScale','yScale']
keys=['cellAttr','xScale','yScale']
print(wi.params.path)

/content/drive/MyDrive/temp/231212


In [ ]:


for daKey in keys:
    print("Generate ",daKey)
    if "nCells" in daKey:
      varItersList = [30,40,50,60,70,80,90]
    elif "cellAttr" in daKey:
      varItersList = [0.5, 0.75 , 1.0, 2.5, 5.0]
    elif "xScale" in daKey:
      #varItersList = [0,1.,2.,3.,4.,5.,6.,7.]
      varItersList = [ 10.,25.,50.,75.,100.]
    elif "yScale" in daKey:
      #varItersList = [0,1.,2.,3.,4.,5.,6.,7.]
      varItersList = [-5, 10.,25.,50.,75.,100.]
    else:
      varItersList = varIter
    writeFile = nocrowdersatp['outName']+"_"+daKey+".bash"
    wi.WriteIterativeYaml(nocrowdersatp,daKey,varIter=varItersList, varRuns=varRuns,
                         writeFile=writeFile)



Generate  cellAttr
Wrote nocrowder_atp_cellAttr.bash
Generate  xScale
Wrote nocrowder_atp_xScale.bash
Generate  yScale
Wrote nocrowder_atp_yScale.bash


In [ ]:
#bp.processYamls(nocrowdersatp['outName'],
#                yamlNamePrefix=nocrowdersatp['outName'],prefixOptions=keys,
#                display=False
#               )

## crowder  case
### No ATP

In [11]:
crowders = daVars.copy()
crowders['outName']='crowder'
#keys=['nCrowders','crowderRad','crowderAttr']
keys=['crowderAttr']
print(wi.params.path)

/content/drive/MyDrive/temp/231212


In [12]:
crowders_repulsive = crowders.copy()
crowders_repulsive['cellChg']=-2
crowders_repulsive['crowderChg']= 2
crowders_repulsive['outName']= 'crowders_repulsive'
key=['crowderChg']
varItersList = [1,2,3] # only one conc. makes sense ,1.,2.,3.,4.,5.,6.,7.]

daKey = key[0]
print("Generate ",daKey)
writeFile = crowders_repulsive['outName']+"_"+daKey+".bash"
wi.WriteIterativeYaml(crowders_repulsive,daKey,varIter=varItersList, varRuns=varRuns,
                      writeFile=writeFile)

Generate  crowderChg
Wrote crowders_repulsive_crowderChg.bash


In [ ]:
for daKey in keys:
    print("Generate ",daKey)
    writeFile = crowders['outName']+"_"+daKey+".bash"
    if daKey == 'nCrowders':
      varItersList = [1,4,9,16,18]
    elif daKey == 'crowderRad':
      varItersList = [0.1,1,3,5,7,9,10]
    elif daKey == 'crowderAttr':
      #varItersList = [0.1,1,3,5,7,9,10]
      varItersList = [10,25,50,75,100]
    else:
      varItersList= varIter
    wi.WriteIterativeYaml(crowders,daKey,varIter=varItersList, varRuns=varRuns,
                         writeFile=writeFile)

Generate  crowderAttr
Wrote crowder_crowderAttr.bash


In [ ]:
#bp.processYamls(crowders['outName'],yamlNamePrefix=crowders['outName'],prefixOptions=keys)

### +ATP

In [ ]:
crowdersatp = daVars.copy()
crowdersatp['outName']='crowder_atp'
crowdersatp['xPotential']=True
keys=['nCrowders','crowderRad','crowderAttr']
keys=['crowderAttr']
print(wi.params.path)

/content/drive/MyDrive/temp/231212


In [ ]:
for daKey in keys:
    print("Generate ",daKey)
    writeFile = crowdersatp['outName']+"_"+daKey+".bash"
    if daKey == 'nCrowders':
      varItersList = [1,4,9,16,18]
    elif daKey == 'crowderRad':
      varItersList = [0.1,1,3,5,7,9,10]
    elif daKey == 'crowderAttr':
      #varItersList = [0.1,1,3,5,7,9,10]
      varItersList = [10,25,50,75,100]
    #elif "xScale" in daKey:
    #  varItersList = [0,1.,2.,3.,4.,5.,6.,7.]
    else:
      varItersList = 7
    wi.WriteIterativeYaml(crowdersatp,daKey,varIter=varItersList, varRuns=varRuns,
                         writeFile=writeFile)

Generate  nCrowders
Wrote crowder_atp_nCrowders.bash
Generate  crowderRad
Wrote crowder_atp_crowderRad.bash
Generate  crowderAttr
Wrote crowder_atp_crowderAttr.bash


In [ ]:
# bp.processYamls(crowdersatp['outName'],yamlNamePrefix=crowdersatp['outName'],prefixOptions=keys)

## Move bash files

In [ ]:
!ln -s /content/drive/MyDrive/temp/231212/ /content/231212

In [ ]:
!cp *sh /content/231212/

In [ ]:
!ls -lrt

total 36
drwxr-xr-x 1 root root 4096 Dec 28 14:19 sample_data
drwxr-xr-x 7 root root 4096 Jan  2 21:35 cellmigration
drwx------ 5 root root 4096 Jan  2 21:36 drive
-rw-r--r-- 1 root root 1425 Jan  2 21:43 nocrowder_cellAttr.bash
-rw-r--r-- 1 root root 1525 Jan  2 21:48 nocrowder_atp_cellAttr.bash
-rw-r--r-- 1 root root 1505 Jan  2 21:48 nocrowder_atp_xScale.bash
-rw-r--r-- 1 root root 1805 Jan  2 21:48 nocrowder_atp_yScale.bash
-rw-r--r-- 1 root root 1480 Jan  2 21:50 crowder_crowderAttr.bash
-rw-r--r-- 1 root root 1580 Jan  2 21:51 crowder_atp_crowderAttr.bash
lrwxrwxrwx 1 root root   35 Jan  2 22:05 231212 -> /content/drive/MyDrive/temp/231212/
